In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm,inch
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
pdfmetrics.registerFont(UnicodeCIDFont('STSong-Light'))
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import BaseDocTemplate, Frame, PageTemplate, Paragraph, Spacer, Table, LongTable, TableStyle, tableofcontents, PageBreak, Image
from reportlab.lib.enums import TA_JUSTIFY, TA_LEFT, TA_CENTER
from reportlab.lib import colors


In [2]:
class BaseReport(object):
    def __init__(self,name):
        pdfmetrics.registerFont(TTFont('SimSun', 'SimSun.ttf'))  # 默认不支持中文，需要注册字体
        pdfmetrics.registerFont(TTFont('SimSunBd', 'SimSun-bold.ttf'))
        
        self.doc = BaseDocTemplate("{}.pdf".format(name), topMargin = 3.5*cm)    #声明一个文档模版类，filename就是存放pdf的地址，
        self.frames = {}  # 声明一个字典，用于存放所有的frame
        self.create100_frame()
        self.page_templates = self.create_page_template()  # 声明一个列表，用于存放所有的页面模板数据
        self.add_page_template()
        self.Elements = []  # 声明一个列表，用于存放所有元素
        
        self.stylesheet = getSampleStyleSheet()   # 获取样式集
        self.style_list = {}  # 声明一个字典，用户存放自定义的字体样式
        self.get_style()
        
    def get_style(self):
        """
        获取样式
        """
        name_items = {
            "BodyText":{"fontSize":12,"textColor":"#FFFF00","spaceAfter":1},
            "Heading1":{"fontSize":12,"textColor":"black","spaceAfter":1},
            "Normal":{"fontSize":12,"textColor":"#FFFF00","spaceAfter":1},
            "BodyText":{"fontSize":12,"textColor":"#0000000","spaceAfter":1},
            "Italic":{"fontSize":12,"textColor":"#FFFF00","spaceAfter":1},
            "Title":{"fontSize":28,"textColor":"black","spaceAfter":30},
            "Heading2":{"fontSize":12,"textColor":"black","spaceAfter":1},
            "Heading3":{"fontSize":12,"textColor":"black","spaceAfter":1},
            "Code":{"fontSize":12,"textColor":"#FFFF00","spaceAfter":1},
        }
        for name,par in name_items.items():
            st = self.stylesheet[name]
            st.textColor = par["textColor"]
            st.fontSize = par["fontSize"]
            st.spaceAfter = par["spaceAfter"]
            st.fontName = 'SimSun'
            self.style_list[name] = st
        
        
        
    def add_style_sheet(self,name):
        """
        添加样式
        """
        style = ParagraphStyle(name=name,fontName="SimSun",fontSize=10,textColor='black',leading=20,spaceBefore=0,spaceAfter=10,
                               leftIndent=0,rightIndent=0,firstLineIndent=20,alignment=TA_JUSTIFY,
                   # bulletFontSize=15,       #bullet为项目符号相关的设置
                   # bulletIndent=-50,
                   # bulletAnchor='start',
                   # bulletFontName='Symbol'
                   )
        self.style_list[name] = style
       
    
    def footer(self,canvas, doc):
        """
        设置页脚
        :param canvas:Canvas类型  pdf画布
        :param doc:doc类型   整个pdf文件
        """
        canvas.saveState()                                                                                    #先保存当前的画布状态
        pageNumber = ("第%s页" %canvas.getPageNumber())                                                           #获取当前的页码
        p = Paragraph(pageNumber, self.style_list["BodyText"])
        w, h = p.wrap(3*cm, 1*cm)                                                                             #申请一块1cm大小的空间，返回值是实际使用的空间
        # p.drawOn(canvas, foot_coordinate_x, foot_coordinate_y)                                                #将页码放在指示坐标处
        p.drawOn(canvas, 300, 10)
        canvas.restoreState()
        
    def header(self,canvas, doc):
        """
        设置页眉
        :param canvas:Canvas类型  pdf画布
        :param doc:doc类型     整个pdf文件
        """
        canvas.saveState()
        p = Paragraph("<img src='%s' width='%d' height='%d'/>" %("./材料v1.0.0/1.jpg", 80, 45), self.style_list["BodyText"])  #使用一个Paragraph Flowable存放图片
        w, h = p.wrap(doc.width, doc.bottomMargin)
        p.drawOn(canvas, doc.leftMargin, doc.topMargin + doc.height - 0.5*cm)                                                #放置图片
        p = Paragraph("<font size=10>广州达安临床检验中心 Guangzhou Da An Clinical Laboratory Center</font>", self.style_list["BodyText"])
        w,h = p.wrap(doc.width, doc.bottomMargin)
#         p.drawOn(canvas, doc.leftMargin+doc.width-2.2*cm, doc.topMargin+ doc.height-0.3*cm) #放置报告这句话
        p.drawOn(canvas, doc.leftMargin + 3.2*cm, doc.topMargin+ doc.height-0.3*cm)
        canvas.line(doc.leftMargin, doc.bottomMargin+doc.height + 0.5*cm, doc.leftMargin+doc.width, doc.bottomMargin+doc.height + 0.5*cm) #画一条横线
        canvas.restoreState()
        
    def create100_frame(self):
        """
        生成一个100%的frame
        return:frame
        """
        frame100 = Frame(self.doc.leftMargin, self.doc.bottomMargin, 
                         self.doc.width, self.doc.height, id='normal')   #声明一块Frame，存放页码
        self.frames["frame100"] = frame100
    
    def create_page_template(self):
        """
        生成页面模板
        """
        templates = []
        for k,v in self.frames.items():
            templates.append(PageTemplate(id=k,frames=v,onPage=self.header, onPageEnd=self.footer))
        return templates
    
    def add_empty_line(self):
        """添加一行空行"""
        self.Elements.append(Paragraph("", self.style_list["BodyText"]))
    
    def add_page_template(self):
        """
        添加页面模板
        """
        self.doc.addPageTemplates(self.page_templates)
    
    def next_page(self):
        """
        翻到下一页
        """
        self.Elements.append(PageBreak())
        
    def create_doc(self):
        """
        创建报告文件
        """
        self.doc.build(self.Elements)
        
    def run(self):
        """
        生成报告文件
        """
        pass
        
    
        
        

In [5]:
class IntestinalFloraReport(BaseReport):
    """肠道菌群报告"""
    def __init__(self,name):
        super().__init__(name)
        
    def add_base_info_table(self):
        """添加一个基础信息表格"""
        table_data = [
                    ['编号：', 'XXXXXXXXXXXXX', '姓名：','XXX'],
                    ['性别：', '男', '年龄：','XX'],
                    ['样本来源：', 'XX省XX市XX区XX街道XX社区居委会\nXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX', '样本类型：','粪便'],
                    ['检测方法：', '下一代高通量测序技术\n（Illuminna）', '检测单位：','XX医院检验医学部'],
                    ['检测日期：2021.11', '', '', ''],
              ]
        table_style = [
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('FONTSIZE', (0, 1), (-1, -1), 10),  # 字体大小
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
            ("BOTTOMPADDING",(0, 0), (1, 1), 5),
            ('SPAN', (0, 4), (3, 4)),  # 合并
            ('BACKGROUND', (0, 0), (0, 3), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (1, 0), (1, 3), '#f0f0f0'),  # 设置背景颜色
            ('BACKGROUND', (2, 0), (2, 3), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (3, 0), (3, 3), '#f0f0f0'),  # 设置背景颜色
            ('BACKGROUND', (0, 4), (3, 4), '#e1e1e1'),  # 设置背景颜色
#             ('TEXTCOLOR', (0, -1), (0, -1), colors.green),  # 设置表格内文字颜色
            ('GRID', (0, 0), (-1, -1), 1.5, colors.white),  # 设置表格框线为灰色，线宽为0.1
            ('ALIGN', (0, 4), (3, 4) , 'RIGHT'),
            ('BOTTOMMARGING',(0, 0), (-1, -1),10)
        ]
        table = Table(data=table_data, style=table_style)
        return table

    def run(self):
        # 插入大标题
        title = '肠道菌群检测报告'
        self.Elements.append(Paragraph(title, self.style_list["Title"]))
        # 添加一个基础信息表格
        table = self.add_base_info_table()
        self.Elements.append(table)
         # 添加一个空行
        title = ''
        self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
         # 添加一个空行
        title = ''
        self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
        # 添加一个标题一的图片
        image = Image("./2.jpg",hAlign="LEFT")
        image.drawWidth = 250
        image.drawHeight = 40
        self.Elements.append(image)
         # 添加一个空行
        title = ''
        self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
         # 添加一个空行
        title = ''
        self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
        # 添加一个处理结果的图片
        image = Image("./3.jpg",hAlign="CENTER")
        image.drawWidth = 450
        image.drawHeight = 150
        self.Elements.append(image)
        title = ''
        self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
         # 添加一个空行
        # 添加一段文字说明
        contents = '评价: 高于4.36%的人群-您的微生态多样性属于中等偏低水平,提示您的肠道菌群多样性 \n 不高。肠道菌群丰富度高,是健康肠道菌群的重要特征。'
        self.Elements.append(Paragraph(contents, self.style_list["BodyText"]))

        
        pass

In [7]:
ifR = IntestinalFloraReport("test1")
ifR.run()
ifR.create_doc()

In [19]:
def wrapper(cls_):
    cls_.__tablename__ = "a_" + cls_.__tablename__
    return cls_

In [20]:
class Base(object):
    
    def __init__():
        print("A")

@wrapper
class A(Base):
    __tablename__ = "a"

In [21]:
A.__tablename__

'a_a'